## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

# Extended Events

> -  (Query 1)
> -  (Query 2)
> -  (Query 3)
> -  (Query 4)

> - Active Sessions in Azure

In [ ]:
--View active Extended events in Azure 
SELECT * FROM sys.dm_xe_database_sessions

> - Blocked Processes - Brent Ozar https://www.brentozar.com/archive/2014/03/extended-events-doesnt-hard

In [ ]:
--Blocked processes extended event
--DROP EVENT SESSION [blocked_process] ON SERVER 
GO

CREATE EVENT SESSION [blocked_process] ON SERVER
ADD EVENT sqlserver.blocked_process_report(
    ACTION(sqlserver.client_app_name,
           sqlserver.client_hostname,
           sqlserver.database_name)) ,
ADD EVENT sqlserver.xml_deadlock_report (
    ACTION(sqlserver.client_app_name,
           sqlserver.client_hostname,
           sqlserver.database_name))
ADD TARGET package0.asynchronous_file_target
(SET filename = N'D:\blocked_process.xel',
     metadatafile = N'D:\blocked_process.xem',
     max_file_size=(65536),
     max_rollover_files=5)
WITH (MAX_DISPATCH_LATENCY = 5SECONDS)
GO
 

In [ ]:
--Set up blocking threshold
/* Make sure this path exists before you start the trace! */

EXEC sp_configure 'show advanced options', 1 ;
GO
RECONFIGURE ;
GO
/* Enabled the blocked process report */
EXEC sp_configure 'blocked process threshold', '5';
RECONFIGURE
GO
/* Start the Extended Events session */
ALTER EVENT SESSION [blocked_process] ON SERVER
STATE = START;

EXEC sp_configure 'show advanced options', 1 ;
GO
RECONFIGURE ;
GO
/* Enabled the blocked process report */
EXEC sp_configure 'blocked process threshold', '5';
RECONFIGURE
GO


In [ ]:
/* Start the Extended Events session */
ALTER EVENT SESSION [blocked_process] ON SERVER
STATE = START;


In [ ]:
/*View blocked processes from Extended Event*/
WITH events_cte AS (
  SELECT
    xevents.event_data,
    DATEADD(mi,
    DATEDIFF(mi, GETUTCDATE(), CURRENT_TIMESTAMP),
    xevents.event_data.value(
      '(event/@timestamp)[1]', 'datetime2')) AS [event time] ,
    xevents.event_data.value(
      '(event/action[@name="client_app_name"]/value)[1]', 'nvarchar(128)')
      AS [client app name],
    xevents.event_data.value(
      '(event/action[@name="client_hostname"]/value)[1]', 'nvarchar(max)')
      AS [client host name],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="database_name"]/value)[1]', 'nvarchar(max)')
      AS [database name],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="database_id"]/value)[1]', 'int')
      AS [database_id],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="object_id"]/value)[1]', 'int')
      AS [object_id],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="index_id"]/value)[1]', 'int')
      AS [index_id],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="duration"]/value)[1]', 'bigint') / 1000
      AS [duration (ms)],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="lock_mode"]/text)[1]', 'varchar')
      AS [lock_mode],
    xevents.event_data.value(
      '(event[@name="blocked_process_report"]/data[@name="login_sid"]/value)[1]', 'int')
      AS [login_sid],
    xevents.event_data.query(
      '(event[@name="blocked_process_report"]/data[@name="blocked_process"]/value/blocked-process-report)[1]')
      AS blocked_process_report,
    xevents.event_data.query(
      '(event/data[@name="xml_report"]/value/deadlock)[1]')
      AS deadlock_graph
  FROM    sys.fn_xe_file_target_read_file
    ('D:\blocked_process*.xel',
     'D:\blocked_process*.xem',
     null, null)
    CROSS APPLY (SELECT CAST(event_data AS XML) AS event_data) as xevents
)
SELECT
  CASE WHEN blocked_process_report.value('(blocked-process-report[@monitorLoop])[1]', 'nvarchar(max)') IS NULL
       THEN 'Deadlock'
       ELSE 'Blocked Process'
       END AS ReportType,
  [event time],
  CASE [client app name] WHEN '' THEN ' -- N/A -- '
                         ELSE [client app name]
                         END AS [client app _name],
  CASE [client host name] WHEN '' THEN ' -- N/A -- '
                          ELSE [client host name]
                          END AS [client host name],
  [database name],
  COALESCE(OBJECT_SCHEMA_NAME(object_id, database_id), ' -- N/A -- ') AS [schema],
  COALESCE(OBJECT_NAME(object_id, database_id), ' -- N/A -- ') AS [table],
  index_id,
  [duration (ms)],
  lock_mode,
  COALESCE(SUSER_NAME(login_sid), ' -- N/A -- ') AS username,
  CASE WHEN blocked_process_report.value('(blocked-process-report[@monitorLoop])[1]', 'nvarchar(max)') IS NULL
       THEN deadlock_graph
       ELSE blocked_process_report
       END AS Report
FROM events_cte
ORDER BY [event time] DESC ;






- #1 Capture procedures greater than N seconds. On-prem

In [ ]:
--TRUNCATE TABLE  DBA.dbo.xEventData

WITH x
AS (SELECT n.value('(@name)[1]', 'varchar(50)') AS event_name,
           n.value('(@package)[1]', 'varchar(50)') AS package_name,
           n.value('(@timestamp)[1]', 'datetime2') AS [utc_timestamp],
           n.value('(data[@name="object_name"]/value)[1]', 'varchar(100)') AS sql_text,
           n.value('(data[@name="duration"]/value)[1]', 'bigint') AS duration,
           --n.value('(data[@name="physical_reads"]/value)[1]', 'int') as physical_reads,
           --n.value('(data[@name="logical_reads"]/value)[1]', 'int') as logical_reads,
           --n.value('(data[@name="writes"]/value)[1]', 'int') as writes,
           --n.value('(data[@name="row_count"]/value)[1]', 'int') as row_count,
           --n.value('(data[@name="last_row_count"]/value)[1]', 'int') as last_row_count,
           --n.value('(data[@name="line_number"]/value)[1]', 'int') as line_number,
           --n.value('(data[@name="offset"]/value)[1]', 'int') as offset,
           --n.value('(data[@name="offset_end"]/value)[1]', 'int') as offset_end,
           n.value('(data[@name="statement"]/value)[1]', 'nvarchar(max)') AS statement,
           n.value('(action[@name="database_name"]/value)[1]', 'nvarchar(128)') AS database_name
    FROM
    (
        SELECT CAST(event_data AS XML) AS event_data
        FROM sys.fn_xe_file_target_read_file(
                                                
												--'C:\TEMP\xEvents\CrossDockExport_Tracker_For_Beth*.xel',
												--'C:\TEMP\xEvents\DBA_shortSPs*.xel',
												'D:\Databases\MSSQL12.MSSQLSERVER\MSSQL\Log\DBA_LongRunningSPs*.xel',
                                                NULL,
                                                NULL,
                                                NULL
                                            )
    ) ed --APLFS145

        --from sys.fn_xe_file_target_read_file('C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\Log\DBA_LongRunningSPs*.xel', null, null, null)) ed --APLFS184

        CROSS APPLY ed.event_data.nodes('event') AS q(n) )
--INSERT	INTO DBA.dbo.xEventData

SELECT  x.event_name,
       x.package_name,
       x.utc_timestamp,
       x.sql_text,
       CAST(x.duration AS DECIMAL(38, 10)) / 1000000 AS DurationInSeconds,
       x.statement,
       x.database_name
INTO DBA.dbo.xEventData
FROM x
--WHERE sql_text='csp_open_orders_out_V2'
;


SELECT * FROM DBA.dbo.xEventData AS xed

SELECT database_name,
       sql_text,
       COUNT(sql_text) AS Executions,
       AVG(DurationInSeconds) AS avgDuration,
	   statement
FROM DBA.dbo.xEventData
--WHERE utc_timestamp between '2021-02-08 11:30:00' AND '2021-02-09 11:30:34'
WHERE statement LIKE '%msp_report_pallets_live_by_carrier%'
GROUP BY sql_text,
         database_name,
		 statement
ORDER BY Executions DESC;


SELECT  TOP 100 database_name,
       *,
	   statement
FROM DBA..xEventData
--WHERE utc_timestamp between '2021-02-08 11:30:00' AND '2021-02-09 11:30:34'
WHERE statement LIKE '%msp_compliance_details_raw_by_mda_export%'
ORDER BY utc_timestamp  desc

In [ ]:
--DROP EVENT SESSION [DBA_LongRunningSPs] ON SERVER 
GO

CREATE EVENT SESSION [DBA_LongRunningSPs] ON SERVER 
ADD EVENT sqlserver.rpc_completed(
    ACTION(sqlserver.database_name,sqlserver.plan_handle,sqlserver.sql_text)
    WHERE ([package0].[greater_than_uint64]([duration],(25000000)) AND [sqlserver].[database_id]>(3)))
ADD TARGET package0.event_file(SET filename=N'D:\DBA_LongRunningSPs.Xel',max_file_size=(5),max_rollover_files=(4))
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=ON,STARTUP_STATE=ON)
GO
ALTER EVENT SESSION [DBA_LongRunningSPs]    
ON SERVER
--ON DATABASE
STATE = STOP;   -- STOP;

> - Capture specific porcedure details. On-prem
>   - rpc_completed
>   - sp_statement_completed
>   - sql_batch_completed



In [ ]:
--Capture procedure details rpc_completed, sp_completed, sql_batch_completed for specified procedure 
CREATE EVENT SESSION [Capture procedure details] ON SERVER 
ADD EVENT sqlserver.rpc_completed(SET collect_data_stream=(1)
    ACTION(sqlserver.database_id,sqlserver.database_name,sqlserver.server_principal_name,sqlserver.session_id)
    WHERE ([package0].[greater_than_uint64]([sqlserver].[database_id],(4)) AND [package0].[equal_boolean]([sqlserver].[is_system],(0)) 
    AND [sqlserver].[like_i_sql_unicode_string]([sqlserver].[sql_text],N'%ProcedureName%'))),
ADD EVENT sqlserver.sp_statement_completed(
    ACTION(sqlserver.database_id,sqlserver.database_name,sqlserver.server_principal_name,sqlserver.session_id)
    WHERE ([package0].[greater_than_uint64]([sqlserver].[database_id],(4)) AND [package0].[equal_boolean]([sqlserver].[is_system],(0)) 
    AND [sqlserver].[like_i_sql_unicode_string]([sqlserver].[sql_text],N'%ProcedureName%'))),
ADD EVENT sqlserver.sql_batch_completed(
    ACTION(sqlserver.database_id,sqlserver.database_name,sqlserver.server_principal_name,sqlserver.session_id)
    WHERE ([package0].[greater_than_uint64]([sqlserver].[database_id],(4)) AND [package0].[equal_boolean]([sqlserver].[is_system],(0)) 
    AND [sqlserver].[like_i_sql_unicode_string]([sqlserver].[sql_text],N'%ProcedureName%')))
ADD TARGET package0.event_file(SET FILENAME=N'Capture procedure details',max_file_size=(5))
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=ON,STARTUP_STATE=ON)

> - Extended Events duration differ based on the type. This query details the type 
> - Source  [https://itectec.com/database/sql-server-extended-event-duration-millisecond-or-microsecond](https://itectec.com/database/sql-server-extended-event-duration-millisecond-or-microsecond)



In [ ]:
-- View duration for each datatype as it can be milli or microseconds
SELECT p.name package_name,
       o.name event_name,
       c.name event_field,
       DurationUnit= CASE
                         WHEN c.description LIKE '%milli%' 
                         THEN SUBSTRING(c.description, CHARINDEX('milli', c.description),12)
                         WHEN c.description LIKE '%micro%' 
                         THEN SUBSTRING(c.description, CHARINDEX('micro', c.description),12)
                         ELSE NULL
                     END,
       c.type_name field_type,
       c.column_type column_type
FROM sys.dm_xe_objects o
JOIN sys.dm_xe_packages p ON o.package_guid = p.guid
JOIN sys.dm_xe_object_columns c ON o.name = c.object_name
WHERE o.object_type = 'event'
AND c.name ='duration'

> - Monitor UDF Sessions
>    - Areas of concern, with scalar functions


In [ ]:
--Extended event to trace scalar function useage
CREATE EVENT SESSION [UDF_Trace] ON SERVER 
ADD EVENT sqlserver.module_end(
ACTION(sqlserver.database_name,sqlserver.sql_text)
WHERE ([sqlserver].[equal_i_sql_ansi_string]([object_type],'FN') 
AND [sqlserver].[database_name]=N'GoSTock')) 
ADD TARGET package0.histogram(SET filtering_event_name=N'sqlserver.module_end',
source=N'object_name',source_type=(0))
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,
MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,
MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=OFF,STARTUP_STATE=OFF)
GO
--to view summarised data go Extended Event 
	--UDF_Trace
			--package0.histogram / View Target Data

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)